# Generate disk-based hashed similarities


In [ ]:
import os
import sys

def find_project_root(target_folder="masteroppgave"):
    """Find the absolute path of a folder by searching upward."""
    currentdir = os.path.abspath("__file__")  # Get absolute script path
    while True:
        if os.path.basename(currentdir) == target_folder:
            return currentdir  # Found the target folder
        parentdir = os.path.dirname(currentdir)
        if parentdir == currentdir:  # Stop at filesystem root
            return None
        currentdir = parentdir  # Move one level up

# Example usage
project_root = find_project_root("masteroppgave")

if project_root:
    sys.path.append(project_root)
    print(f"Project root found: {project_root}")
else:
    raise RuntimeError("Could not find 'masteroppgave' directory")


from computation.disk_similarity import generate_disk_hash_similarity

# Rome


### DTW


In [ ]:
measure = "dtw"
similarities, hashes = generate_disk_hash_similarity(
    city="rome", diameter=0.6, layers=5, disks=50, measure=measure, size=50
)

# print only the elements in hashes that has several elements within each key
for key, value in hashes.items():
    if len(value) > 1:
        print(key, value)

 
output_path = f"similarity_values/disk/rome/disk_rome-{measure}.csv"
# similarities.to_csv(os.path.abspath(output_path))

### Frechet


In [ ]:
measure = "frechet"
similarities = generate_disk_hash_similarity(
    city="rome", diameter=1.6, layers=5, disks=100, measure=measure, size=500
)
output_path = f"similarity_values/disk/rome/disk_rome-{measure}.csv"
# similarities.to_csv(os.path.abspath(output_path))

# Porto


In [ ]:
measure = "dtw"
similarities = generate_disk_hash_similarity(
    "porto", diameter=2.2, layers=4, disks=60, measure=measure, size=50
)
output_path = f"similarity_values/disk/porto/disk_porto-{measure}.csv"
similarities.to_csv(os.path.abspath(output_path))